In [1]:
import numpy as np
import torch
import pickle
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import random

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
pip install fbm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [4]:
from fbm import FBM
import numpy as np

In [5]:
grid_points = 128
num_of_train = 50000 
batch = 1


In [6]:
# Generating H randomly and then creating fGn-s
# %%time
# train_data = []


# num_test = num_of_train
# rand_test = []

# for j in range(num_test):
#   H = random.uniform(0.01,1.0)
  
#   f = FBM(n=grid_points, hurst=H, length=1, method='daviesharte')
  

#   train_data.append((f.fgn(), H))


In [7]:
# H generated from 0.05 grid, behaves better for training
# %%time

# train_data = []

# for H in np.arange(0.05,1.0,0.05):
#   f = FBM(n=grid_points, hurst=H, length=1, method='daviesharte')
#   for i in range(500):
#     train_data.append((f.fgn(), H))

# from random import shuffle
# shuffle(train_data)

In [8]:
# Training data, well behaved H
with open('notrandom_regression_train_data_grid128', 'rb') as fp:
    train_data = pickle.load(fp)

In [9]:
# Testing data, random H
with open ('regression_train_data_grid128', 'rb') as fp:
    save = pickle.load(fp)

In [32]:
test_loader = torch.utils.data.DataLoader(save[:50000], batch_size=batch, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch, shuffle=True)

In [13]:
p=0.4
class CNN_simplistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(grid_points, 256)
        self.fc2 = nn.Linear(256, 190)
        self.fc3 = nn.Linear(190, 190)
        self.fc4 = nn.Linear(190, 1)
        self.dropout = nn.Dropout(p)

    def forward(self, x):

        x = self.fc1(x)
        x = F.relu(x)
        
        #fc layer 2
        x = self.fc2(x)
        x = F.relu(x)

        #fc layer 3
        x = self.fc3(x)
        x = F.relu(x)

        x=self.fc4(x)
        
        return x

In [23]:
from torchvision import datasets, transforms
from tqdm.notebook import tqdm, trange

## Training
# Instantiate model  
model_simp = CNN_simplistic().double().to(device) 


# model = MNIST_CNN().to(device) 

# Loss and Optimizer
base_optimizer = torch.optim.SGD
optimizer = torch.optim.Adam(model_simp.parameters(), lr=0.001)
criterion = nn.L1Loss()




# Iterate through train set minibatchs 
for epoch in trange(5):  
    for images, labels in tqdm(train_loader):
        images = images[:,None,:].to(device)
        labels = labels.to(device)
        # Zero out the gradients
        optimizer.zero_grad()

        # ADAM
        # Forward pass
        x = images
        y = model_simp(x)
        loss = criterion(torch.squeeze(y), torch.squeeze(labels))


        
        # Backward pass
        loss.backward()
        optimizer.step()

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/9500 [00:00<?, ?it/s]

  0%|          | 0/9500 [00:00<?, ?it/s]

  0%|          | 0/9500 [00:00<?, ?it/s]

  0%|          | 0/9500 [00:00<?, ?it/s]

  0%|          | 0/9500 [00:00<?, ?it/s]

In [33]:
## Testing
allowed_error = 0.04
norm = 0
total = num_of_train*2//10
counter = 0
correct=0

with torch.no_grad():
    # Iterate through test set minibatchs 
    for images, labels in tqdm(test_loader):

        counter += 1

        images = images[:,None,:].to(device)
        labels = labels.to(device)

        # Forward pass
        x = images  
        y = model_simp(x)
        # print(torch.abs(labels-y))
        correct += torch.sum((abs(labels-y)<allowed_error).float())

print('Accuracy: {}'.format(correct/batch/counter))

  0%|          | 0/50000 [00:00<?, ?it/s]

Accuracy: 0.9193999767303467


In [35]:

f = FBM(n=grid_points, hurst=0.675, length=1, method='daviesharte')
g =f.fgn()

t = torch.tensor((g))[None,:].to(device)
model_simp(t)

tensor([[0.6688]], device='cuda:0', dtype=torch.float64,
       grad_fn=<AddmmBackward0>)